## 1. Project Setup: Create Directories

First, we'll create the basic folder structure for our application. We need a `backend` folder to hold our Python server code and a `frontend` folder for the HTML, CSS, and JavaScript files.

In [1]:
%%bash
mkdir -p /content/vehicle-counter/backend
mkdir -p /content/vehicle-counter/frontend

### Backend Code: The SORT Tracker

This cell writes the Python code for the **SORT (Simple Online and Realtime Tracking)** algorithm into a file named `sort_tracker.py`. This script is a core component of our backend; it's responsible for assigning a consistent ID to each detected vehicle across multiple frames, allowing us to track its movement.

In [2]:
%%writefile /content/vehicle-counter/backend/sort_tracker.py
"""
This module provides an implementation of the SORT (Simple Online and Realtime Tracking)
algorithm, a pragmatic approach for multi-object tracking with a focus on simplicity and speed.

The algorithm uses a Kalman filter for motion prediction and the Hungarian algorithm for data
association. It's designed to be effective for tracking objects like pedestrians or vehicles in
video streams.

Core Components:
- KalmanBoxTracker: Manages the state of a single tracked object using a Kalman filter.
- Sort: The main class that orchestrates the tracking process across multiple objects and frames.
- Helper functions: For calculating Intersection over Union (IoU) and associating detections
  to trackers.
"""
from typing import List, Tuple
import numpy as np
from filterpy.kalman import KalmanFilter
from scipy.optimize import linear_sum_assignment


def iou(bb_test: np.ndarray, bb_gt: np.ndarray) -> float:
    """
    Computes Intersection over Union (IoU) for a single pair of bounding boxes.

    Args:
        bb_test (np.ndarray): Bounding box [x1, y1, x2, y2].
        bb_gt (np.ndarray): Ground truth bounding box [x1, y1, x2, y2].

    Returns:
        float: The IoU value.
    """
    xx1 = max(bb_test[0], bb_gt[0])
    yy1 = max(bb_test[1], bb_gt[1])
    xx2 = min(bb_test[2], bb_gt[2])
    yy2 = min(bb_test[3], bb_gt[3])
    w = max(0.0, xx2 - xx1)
    h = max(0.0, yy2 - yy1)
    inter = w * h
    area1 = max(0.0, (bb_test[2] - bb_test[0])) * max(0.0, (bb_test[3] - bb_test[1]))
    area2 = max(0.0, (bb_gt[2] - bb_gt[0])) * max(0.0, (bb_gt[3] - bb_gt[1]))
    union = area1 + area2 - inter
    return inter / union if union > 0 else 0.0


def iou_batch(dets: np.ndarray, trks: np.ndarray) -> np.ndarray:
    """
    Computes Intersection over Union (IoU) for two sets of bounding boxes in a vectorized manner.

    Args:
        dets (np.ndarray): Detections, shape (N, 4) where N is the number of detections.
        trks (np.ndarray): Tracked objects, shape (M, 4) where M is the number of trackers.

    Returns:
        np.ndarray: An (N, M) matrix with the IoU values.
    """
    if dets.size == 0 or trks.size == 0:
        return np.zeros((dets.shape[0], trks.shape[0]), dtype=np.float32)

    # Vectorized computation of IoU
    dets_exp = dets[:, None, :]
    trks_exp = trks[None, :, :]

    xx1 = np.maximum(dets_exp[..., 0], trks_exp[..., 0])
    yy1 = np.maximum(dets_exp[..., 1], trks_exp[..., 1])
    xx2 = np.minimum(dets_exp[..., 2], trks_exp[..., 2])
    yy2 = np.minimum(dets_exp[..., 3], trks_exp[..., 3])

    w = np.clip(xx2 - xx1, 0.0, None)
    h = np.clip(yy2 - yy1, 0.0, None)
    inter = w * h

    dets_area = np.clip(dets_exp[..., 2] - dets_exp[..., 0], 0.0, None) * np.clip(
        dets_exp[..., 3] - dets_exp[..., 1], 0.0, None
    )
    trks_area = np.clip(trks_exp[..., 2] - trks_exp[..., 0], 0.0, None) * np.clip(
        trks_exp[..., 3] - trks_exp[..., 1], 0.0, None
    )

    union = dets_area + trks_area - inter
    eps = np.finfo(np.float32).eps
    return np.where(union > 0.0, inter / (union + eps), 0.0).astype(np.float32)


def convert_bbox_to_z(bbox: np.ndarray) -> np.ndarray:
    """
    Takes a bounding box in the form [x1, y1, x2, y2] and returns z in the form
    [x, y, s, r] where x,y is the centre of the box, s is the scale/area, and r is
    the aspect ratio.

    Args:
        bbox (np.ndarray): Bounding box in [x1, y1, x2, y2] format.

    Returns:
        np.ndarray: Kalman filter measurement vector [x, y, s, r].
    """
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = bbox[0] + w / 2.0
    y = bbox[1] + h / 2.0
    s = w * h  # scale is just area
    r = w / (h + 1e-6)
    return np.array([x, y, s, r]).reshape((4, 1))


def convert_x_to_bbox(x: np.ndarray) -> np.ndarray:
    """
    Takes a bounding box in the centre form [x, y, s, r] and returns it in the form
    [x1, y1, x2, y2] where x1, y1 is the top-left and x2, y2 is the bottom-right.

    Args:
        x (np.ndarray): Kalman filter state vector [x, y, s, r, ...].

    Returns:
        np.ndarray: Bounding box in [x1, y1, x2, y2] format.
    """
    x_c, y_c, s, r = x[0], x[1], max(1.0, x[2]), max(1e-6, x[3])
    w = np.sqrt(s * r)
    h = s / (w + 1e-6)
    x1 = x_c - w / 2.0
    y1 = y_c - h / 2.0
    x2 = x_c + w / 2.0
    y2 = y_c + h / 2.0
    return np.array([x1, y1, x2, y2]).reshape((1, 4))


class KalmanBoxTracker:
    """
    This class represents the internal state of individual tracked objects observed as bbox.
    It uses a Kalman filter to predict the object's position in subsequent frames.
    """

    _count = 0

    def __init__(self, bbox: np.ndarray):
        """
        Initializes a tracker using initial bounding box.

        The state is [x, y, s, r, dx, dy, ds, dr], where (x, y) is the center,
        s is the scale/area, r is the aspect ratio, and the rest are velocities.
        However, the aspect ratio velocity 'dr' is not used in this model.
        """
        # Define constant velocity model
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        dt = 1.0
        # State Transition Matrix
        self.kf.F = np.array(
            [
                [1, 0, 0, 0, dt, 0, 0],
                [0, 1, 0, 0, 0, dt, 0],
                [0, 0, 1, 0, 0, 0, dt],
                [0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, 0, 1],
            ]
        )
        # Measurement Matrix
        self.kf.H = np.array(
            [
                [1, 0, 0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 1, 0, 0, 0],
            ]
        )
        # Measurement Noise Covariance
        self.kf.R[2:, 2:] *= 10.0
        # Process Covariance
        self.kf.P[4:, 4:] *= 1000.0  # give high uncertainty to the unobservable initial velocities
        self.kf.P *= 10.0
        # Process Noise
        self.kf.Q[-1, -1] *= 0.01
        self.kf.Q[4:, 4:] *= 0.01

        self.kf.x[:4] = convert_bbox_to_z(bbox)

        self.time_since_update = 0
        self.id = KalmanBoxTracker._count
        KalmanBoxTracker._count += 1
        self.hits = 1
        self.hit_streak = 1
        self.age = 0

    def update(self, bbox: np.ndarray):
        """
        Updates the state vector with observed bbox.
        """
        self.time_since_update = 0
        self.hits += 1
        self.hit_streak += 1
        self.kf.update(convert_bbox_to_z(bbox))

    def predict(self) -> np.ndarray:
        """
        Advances the state vector and returns the predicted bounding box estimate.
        """
        self.kf.predict()
        self.age += 1
        if self.time_since_update > 0:
            self.hit_streak = 0
        self.time_since_update += 1
        return convert_x_to_bbox(self.kf.x)

    def get_state(self) -> np.ndarray:
        """
        Returns the current bounding box estimate.
        """
        return convert_x_to_bbox(self.kf.x)


def associate_detections_to_trackers(
    detections: np.ndarray, trackers: np.ndarray, iou_threshold: float
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Assigns detections to tracked object (both represented as bounding boxes).

    Args:
        detections (np.ndarray): A set of detected bounding boxes.
        trackers (np.ndarray): A set of bounding boxes predicted by trackers.
        iou_threshold (float): The IoU threshold for considering a match.

    Returns:
        A tuple of three arrays:
        - matches (np.ndarray): Array of shape (N, 2) with indices of matched (detection, tracker).
        - unmatched_dets (np.ndarray): Array of indices of unmatched detections.
        - unmatched_trks (np.ndarray): Array of indices of unmatched trackers.
    """
    if trackers.size == 0 or detections.size == 0:
        return (
            np.empty((0, 2), dtype=int),
            np.arange(detections.shape[0]),
            np.arange(trackers.shape[0]),
        )

    ious = iou_batch(detections, trackers)
    cost = 1.0 - ious

    # Use the Hungarian algorithm (linear_sum_assignment) to find optimal assignments
    det_idx, trk_idx = linear_sum_assignment(cost)

    matches = []
    unmatched_dets = set(range(detections.shape[0]))
    unmatched_trks = set(range(trackers.shape[0]))

    for d, t in zip(det_idx, trk_idx):
        if ious[d, t] >= iou_threshold:
            matches.append([d, t])
            unmatched_dets.discard(d)
            unmatched_trks.discard(t)

    return (
        np.array(matches, dtype=int) if matches else np.empty((0, 2), dtype=int),
        np.array(sorted(list(unmatched_dets)), dtype=int),
        np.array(sorted(list(unmatched_trks)), dtype=int),
    )


class Sort:
    """
    Sort is the main tracking class. It takes object detections for each frame
    and manages the lifecycle of trackers.
    """

    def __init__(self, max_age: int = 20, min_hits: int = 3, iou_threshold: float = 0.3):
        """
        Initializes the Sort tracker.

        Args:
            max_age (int): Maximum number of frames to keep a track alive without new detections.
            min_hits (int): Minimum number of consecutive detections to start a track.
            iou_threshold (float): IoU threshold for matching detections to existing tracks.
        """
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.trackers: List[KalmanBoxTracker] = []
        self.frame_count = 0
        KalmanBoxTracker._count = 0  # Reset tracker ID count

    def update(self, dets: np.ndarray) -> np.ndarray:
        """
        This method is the main update loop. It should be called for each frame.

        Args:
            dets (np.ndarray): A numpy array of detections in the format [[x1,y1,x2,y2,score],...].
                               If no detections are present, it should be an empty array.

        Returns:
            np.ndarray: A numpy array of tracked objects in the format [[x1,y1,x2,y2,track_id],...].
        """
        self.frame_count += 1

        # 1. Predict new locations of existing trackers
        trks = []
        for t in self.trackers:
            pos = t.predict()
            trks.append(pos.reshape(-1))
        trks = np.array(trks) if len(trks) > 0 else np.empty((0, 4))

        # 2. Associate detections with predicted tracker locations
        detection_bboxes = dets[:, :4] if dets.size else np.empty((0, 4))
        matches, unmatched_dets, unmatched_trks = associate_detections_to_trackers(
            detection_bboxes, trks, self.iou_threshold
        )

        # 3. Update matched trackers with new detection info
        for m in matches:
            trk_idx = m[1]
            det_idx = m[0]
            self.trackers[trk_idx].update(dets[det_idx, :4])

        # 4. Create new trackers for unmatched detections
        for i in unmatched_dets:
            self.trackers.append(KalmanBoxTracker(dets[i, :4]))

        # 5. Manage tracker lifecycle and prepare output
        ret = []
        alive_trackers = []
        for t in self.trackers:
            # A track is considered confirmed if it has been updated recently and has a sufficient hit streak.
            # We also include tracks early on (before min_hits frames have passed) to get initial results.
            if (t.time_since_update < 1) and (
                t.hit_streak >= self.min_hits or self.frame_count <= self.min_hits
            ):
                d = t.get_state().reshape(-1)
                ret.append(np.concatenate([d, [t.id]], axis=0))

            # Remove trackers that have been lost for too long
            if t.time_since_update <= self.max_age:
                alive_trackers.append(t)

        self.trackers = alive_trackers

        return np.array(ret) if len(ret) > 0 else np.empty((0, 5))


if __name__ == "__main__":
    # --- Example Usage ---

    # Create an instance of the Sort tracker
    tracker = Sort(max_age=5, min_hits=2, iou_threshold=0.3)

    # Simulate frames from a video
    # Each frame contains a list of detections: [x1, y1, x2, y2, confidence_score]
    simulated_frames = [
        # Frame 1: One object detected
        np.array([[100, 100, 150, 150, 0.9]]),
        # Frame 2: The object moved slightly
        np.array([[105, 105, 155, 155, 0.92]]),
        # Frame 3: Object continues to move, and a new object appears
        np.array([[110, 110, 160, 160, 0.93], [300, 200, 350, 250, 0.88]]),
        # Frame 4: Both objects move
        np.array([[115, 115, 165, 165, 0.94], [305, 205, 355, 255, 0.89]]),
        # Frame 5: First object is missed by the detector
        np.array([[310, 210, 360, 260, 0.90]]),
        # Frame 6: First object is detected again, second one continues
        np.array([[125, 125, 175, 175, 0.91], [315, 215, 365, 265, 0.91]]),
    ]

    print("--- Running SORT Tracker Simulation ---")
    for frame_num, detections in enumerate(simulated_frames):
        print(f"\n--- Frame {frame_num + 1} ---")
        print(f"Detections:\n{detections}")

        # Update the tracker with the new detections
        tracked_objects = tracker.update(detections)

        print(f"Tracked Objects (Output format: [x1, y1, x2, y2, track_id]):\n{tracked_objects}")

Writing /content/vehicle-counter/backend/sort_tracker.py


### Backend Code: The Main Flask Application

This is the heart of our backend. This cell writes the main application logic to `app.py`. This Python script uses the **Flask** framework to create a web server that does the following:
* Loads the **YOLOv8 model** to detect vehicles.
* Uses the **SORT tracker** to track the detected vehicles.
* Processes a video file frame-by-frame, draws bounding boxes, and counts vehicles crossing a virtual line.
* Provides several **API endpoints**:
    * `/video_feed`: Streams the processed video.
    * `/api/counts`: Returns the latest vehicle counts.
    * `/api/metrics`: Provides performance data like FPS.
    * `/api/reset`: Resets the counters.

In [3]:
%%writefile /content/vehicle-counter/backend/app.py
"""
Real-time Vehicle Counting Web Application using YOLOv8 and Flask.

This script launches a Flask web server that performs real-time vehicle detection
and tracking on a video stream. It serves an annotated video feed and provides
API endpoints to retrieve vehicle counts and performance metrics.

Core Functionality:
- Ingests a video file specified by the VIDEO_PATH environment variable.
- Uses a YOLOv8 model for object detection.
- Employs either the SORT or ByteTrack algorithm for object tracking.
- Draws bounding boxes, track IDs, and trails on each frame.
- Counts vehicles as they cross a predefined horizontal line.
- Streams the processed video to a web browser via an MJPEG stream.
- Provides a REST API for accessing counts, metrics, and resetting the state.

Configuration is managed via environment variables. Key variables include:
- VIDEO_PATH: Path to the input video file.
- MODEL_PATH: Path to the YOLOv8 model file (e.g., yolov8s.pt).
- TRACKER_MODE: The tracking algorithm to use, either "SORT" or "BYTE".
- CONF_THRESH: Confidence threshold for object detection.
- LINE_Y: The vertical position of the counting line.
"""
import os
import time
import threading
from collections import deque, defaultdict

import cv2
import numpy as np
import torch
from flask import Flask, Response, jsonify
from flask_cors import CORS
from ultralytics import YOLO

# Assuming sort_tracker.py containing the Sort class is in the same directory.
from sort_tracker import Sort

# --- Configuration from Environment Variables ---
VIDEO_PATH = os.environ.get("VIDEO_PATH", "../content/video1.mp4")
MODEL_PATH = os.environ.get("MODEL_PATH", "yolov8s.pt")
MODEL_DEVICE = os.environ.get('MODEL_DEVICE', 'auto').lower()
CONF_THRESH = float(os.environ.get("CONF_THRESH", 0.35))
IOU_THRESH = float(os.environ.get("IOU_THRESH", 0.5))
DEFAULT_LINE_Y = int(os.environ.get("LINE_Y", 360))
TRAIL_LEN = int(os.environ.get("TRAIL_LEN", 20))
RESIZE_WIDTH = int(os.environ.get("RESIZE_WIDTH", 960))
JPEG_QUALITY = int(os.environ.get("JPEG_QUALITY", 80))
RESET_ON_LOOP = os.environ.get("RESET_ON_LOOP", "false").lower() in {"1", "true", "yes"}
TRACKER_MODE = os.environ.get("TRACKER_MODE", "SORT").upper()

# --- Constants ---
VEHICLE_NAMES = {"car", "truck", "bus", "motorcycle"}
TEXT_COLOR = (255, 255, 255)  # White
LINE_COLOR = (0, 255, 255)   # Cyan
BOX_COLOR = (0, 255, 0)      # Green
ID_COLOR = (255, 0, 0)       # Blue

# --- Application Setup ---
app = Flask(__name__)
# Enable Cross-Origin Resource Sharing for API and video feed routes.
CORS(app, resources={r"/api/*": {"origins": "*"}, r"/video_feed": {"origins": "*"}, r"/health": {"origins": "*"}})

# --- Model and Tracker Initialization ---
# Load the YOLOv8 model from the specified path.
model = YOLO(MODEL_PATH)
# Determine the active device for the model (CUDA or CPU).
if MODEL_DEVICE == 'auto':
    MODEL_DEVICE_ACTIVE = 'cuda' if torch.cuda.is_available() else 'cpu'
else:
    MODEL_DEVICE_ACTIVE = MODEL_DEVICE
# Move the model to the selected device, with a fallback to CPU.
try:
    model.to(MODEL_DEVICE_ACTIVE)
except Exception as exc:
    print(f'[warn] Failed to move model to {MODEL_DEVICE_ACTIVE}: {exc}')
    MODEL_DEVICE_ACTIVE = 'cpu'
    try:
        model.to(MODEL_DEVICE_ACTIVE)
    except Exception as exc_cpu:
        print(f'[warn] Failed to move model to cpu: {exc_cpu}')

# Open the video file for processing.
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f"Could not open video: {VIDEO_PATH}")

# Initialize the SORT tracker if selected.
mot = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

# --- Global State Variables ---
# Dictionary to store vehicle counts, protected by a lock for thread safety.
counts = {"up": 0, "down": 0}
counts_lock = threading.Lock()

# Dictionary to store the history of each tracked object.
track_history = defaultdict(lambda: {
    "centers": deque(maxlen=TRAIL_LEN), # Stores recent center points for drawing trails.
    "last_side": None,                  # 'above' or 'below' the line.
    "last_seen": time.time(),           # Timestamp of the last update.
})

# Dictionary for performance metrics, also protected by a lock.
metrics = {
    "start_time": time.time(),
    "frames": 0,
    "recent_ts": deque(maxlen=120),    # Timestamps of recent frames for FPS calculation.
    "fps": 0.0,
    "inference_ms": deque(maxlen=240), # Inference times for averaging.
    "avg_inference_ms": 0.0,
    "model_device": MODEL_DEVICE_ACTIVE,
}
metrics_lock = threading.Lock()


def preprocess(frame):
    """
    Resizes the frame to a standard width if required.

    Args:
        frame (np.ndarray): The input video frame.

    Returns:
        np.ndarray: The resized frame.
    """
    if RESIZE_WIDTH > 0:
        h, w = frame.shape[:2]
        if w != RESIZE_WIDTH:
            scale = RESIZE_WIDTH / float(w)
            frame = cv2.resize(frame, (RESIZE_WIDTH, int(h * scale)))
    return frame


def update_counts_for_crossing(tid, cy, line_y):
    """
    Updates the up/down counts when a tracked object crosses the line.

    Args:
        tid (int): The track ID of the object.
        cy (int): The current y-coordinate of the object's center.
        line_y (int): The y-coordinate of the counting line.
    """
    info = track_history[tid]
    last_side = info["last_side"]
    # Determine the current side of the line.
    side = "below" if cy > line_y else "above"
    # If this is the first time we see this object, just record its side.
    if last_side is None:
        info["last_side"] = side
        return
    # If the object has crossed the line, update the count.
    if side != last_side:
        direction = "down" if side == "below" and last_side == "above" else "up"
        with counts_lock:
            counts[direction] += 1
        # Update the side for the next frame.
        info["last_side"] = side


def draw_overlays(frame, line_y, tracks):
    """
    Draws all visual annotations onto the frame.

    This includes the counting line, bounding boxes for tracked objects,
    track IDs, object trails, and the current vehicle counts.

    Args:
        frame (np.ndarray): The video frame to draw on.
        line_y (int): The y-coordinate of the counting line.
        tracks (np.ndarray): An array of active tracks from the tracker.

    Returns:
        np.ndarray: The frame with overlays drawn on it.
    """
    h, w = frame.shape[:2]
    # Draw the counting line.
    cv2.line(frame, (0, line_y), (w, line_y), LINE_COLOR, 2)
    cv2.putText(frame, f"Line y={line_y}", (10, max(25, line_y - 8)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, LINE_COLOR, 2, cv2.LINE_AA)
    # Draw boxes, IDs, and trails for each active track.
    for x1, y1, x2, y2, tid in tracks.astype(int):
        cv2.rectangle(frame, (x1, y1), (x2, y2), BOX_COLOR, 2)
        cv2.putText(frame, f"ID {tid}", (x1, max(0, y1 - 6)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, ID_COLOR, 2, cv2.LINE_AA)
        centers = track_history[tid]["centers"]
        # Draw the trail.
        for i in range(1, len(centers)):
            if centers[i - 1] and centers[i]:
                cv2.line(frame, centers[i - 1], centers[i], (200, 200, 200), 2)
    # Draw the total counts on the screen.
    with counts_lock:
        up, down = counts["up"], counts["down"]
    cv2.putText(frame, f"Up: {up}   Down: {down}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, TEXT_COLOR, 2, cv2.LINE_AA)
    return frame


def bump_metrics(inference_ms=None):
    """
    Updates performance metrics such as FPS and average inference time.

    Args:
        inference_ms (float, optional): The inference time for the current frame in milliseconds.
    """
    now = time.time()
    with metrics_lock:
        metrics["frames"] += 1
        metrics["recent_ts"].append(now)
        # Calculate FPS over a sliding window of recent timestamps.
        if len(metrics["recent_ts"]) >= 2:
            dt = metrics["recent_ts"][-1] - metrics["recent_ts"][0]
            if dt > 0:
                metrics["fps"] = (len(metrics["recent_ts"]) - 1) / dt
        # Update average inference time.
        if inference_ms is not None:
            metrics["inference_ms"].append(inference_ms)
            metrics["avg_inference_ms"] = sum(metrics["inference_ms"]) / len(metrics["inference_ms"])


def reset_state(full_reset_tracker=True):
    """
    Resets the application's state, including counts and track history.

    Args:
        full_reset_tracker (bool): If True and using SORT, re-initializes the tracker instance.
    """
    global mot, track_history
    # Reset counts to zero.
    with counts_lock:
        counts["up"] = 0
        counts["down"] = 0
    # Clear the tracking history.
    track_history = defaultdict(lambda: {
        "centers": deque(maxlen=TRAIL_LEN),
        "last_side": None,
        "last_seen": time.time(),
    })
    # Optionally, create a fresh SORT tracker instance.
    if full_reset_tracker and TRACKER_MODE == "SORT":
        mot = Sort(max_age=20, min_hits=3, iou_threshold=0.3)


def frame_generator():
    """
    A generator function that processes video frames and yields them for streaming.

    This is the core processing loop. It reads frames from the video source,
    performs detection and tracking, updates counts, draws overlays, and
    encodes the frame as a JPEG for the MJPEG stream.
    """
    global cap
    while True:
        ret, frame = cap.read()
        # If the video ends, loop back to the beginning.
        if not ret:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            if RESET_ON_LOOP:
                reset_state(full_reset_tracker=True)
            continue

        frame = preprocess(frame)
        line_y = DEFAULT_LINE_Y
        tracks_array = np.empty((0, 5), dtype=np.float32)
        inference_start = time.perf_counter()

        # --- Detection and Tracking Logic ---
        if TRACKER_MODE == "SORT":
            # 1. Run detection.
            results = model(frame, conf=CONF_THRESH, iou=IOU_THRESH, verbose=False)[0]
            dets = []
            if results.boxes is not None and len(results.boxes) > 0:
                # 2. Filter detections for vehicle classes.
                for box in results.boxes:
                    name = results.names[int(box.cls)]
                    if name in VEHICLE_NAMES:
                        xyxy = box.xyxy.cpu().numpy().flatten()
                        p = float(box.conf.cpu())
                        dets.append([xyxy[0], xyxy[1], xyxy[2], xyxy[3], p])
            dets_np = np.array(dets, dtype=np.float32) if dets else np.empty((0, 5), dtype=np.float32)
            # 3. Update SORT tracker with the filtered detections.
            tracks_array = mot.update(dets_np)
        else: # ByteTrack
            # Use YOLO's built-in tracking which uses ByteTrack.
            results = model.track(frame, conf=CONF_THRESH, iou=IOU_THRESH, persist=True, verbose=False)[0]
            byte_tracks = []
            if results.boxes is not None and len(results.boxes) > 0 and results.boxes.id is not None:
                 # Filter tracks for vehicle classes.
                for box in results.boxes:
                    name = results.names[int(box.cls)]
                    if name in VEHICLE_NAMES:
                        xyxy = box.xyxy.cpu().numpy().flatten()
                        tid = int(box.id.cpu())
                        byte_tracks.append([xyxy[0], xyxy[1], xyxy[2], xyxy[3], float(tid)])
            tracks_array = np.asarray(byte_tracks, dtype=np.float32) if byte_tracks else np.empty((0, 5), dtype=np.float32)

        inference_ms = (time.perf_counter() - inference_start) * 1000.0
        now = time.time()

        # --- Update Track History and Counts ---
        active_ids = set()
        for x1, y1, x2, y2, tid in tracks_array:
            tid = int(tid)
            active_ids.add(tid)
            # Calculate the center of the bounding box.
            cx = int((x1 + x2) / 2.0)
            cy = int((y1 + y2) / 2.0)
            # Update the history for this track.
            track_history[tid]["centers"].append((cx, cy))
            track_history[tid]["last_seen"] = now
            # Check if the track crossed the line.
            update_counts_for_crossing(tid, cy, line_y)

        # --- Clean Up Stale Tracks ---
        # Remove tracks that haven't been seen for a few seconds.
        stale_cutoff = now - 5.0
        for tid, meta in list(track_history.items()):
            if meta["last_seen"] < stale_cutoff and tid not in active_ids:
                del track_history[tid]

        # --- Encode and Yield Frame ---
        frame = draw_overlays(frame, line_y, tracks_array)
        # Encode the frame as JPEG.
        ok, jpeg = cv2.imencode(".jpg", frame, [int(cv2.IMWRITE_JPEG_QUALITY), JPEG_QUALITY])
        if not ok:
            continue
        # Update performance metrics.
        bump_metrics(inference_ms)
        # Yield the frame in the format required for MJPEG streaming.
        yield (b"--frame\r\n"
               b"Content-Type: image/jpeg\r\n\r\n" + jpeg.tobytes() + b"\r\n")

# --- Flask API Routes ---
@app.route("/video_feed")
def video_feed():
    """Flask route to serve the MJPEG video stream."""
    return Response(frame_generator(), mimetype="multipart/x-mixed-replace; boundary=frame")


@app.route("/api/counts")
def api_counts():
    """Flask route to get the current vehicle counts as JSON."""
    with counts_lock:
        return jsonify({"up": int(counts["up"]), "down": int(counts["down"])})


@app.route("/api/reset")
def api_reset():
    """Flask route to reset the application's state."""
    reset_state(full_reset_tracker=True)
    return jsonify({"status": "ok", "message": "Counts and state reset."})


@app.route("/api/metrics")
def api_metrics():
    """Flask route to get performance and configuration metrics as JSON."""
    with metrics_lock:
        data = {
            "fps": round(metrics["fps"], 2),
            "avg_inference_ms": round(metrics["avg_inference_ms"], 2),
            "model_device": metrics["model_device"],
            "frames_processed": int(metrics["frames"]),
            "uptime_sec": int(time.time() - metrics["start_time"]),
            "tracker_mode": TRACKER_MODE,
            "reset_on_loop": RESET_ON_LOOP,
            "resize_width": RESIZE_WIDTH,
            "jpeg_quality": JPEG_QUALITY,
            "conf_thresh": CONF_THRESH,
            "iou_thresh": IOU_THRESH,
            "line_y": DEFAULT_LINE_Y,
        }
    return jsonify(data)


@app.route("/health")
def health():
    """Flask route for a simple health check."""
    return jsonify({"ok": True, "service": "vehicle-counter", "tracker_mode": TRACKER_MODE})


@app.route("/")
def root():
    """Flask route for the root URL."""
    return jsonify({"ok": True, "message": "Use /video_feed and /api/counts"})


# --- Main Execution Block ---
if __name__ == "__main__":
    host = os.environ.get("HOST", "0.0.0.0")
    port = int(os.environ.get("PORT", "5000"))
    # Set some default environment variables if they are not already defined.
    # This can be useful for running the script directly without a .env file.
    os.environ.setdefault("RESIZE_WIDTH", "960")
    os.environ.setdefault("JPEG_QUALITY", "80")
    os.environ.setdefault("CONF_THRESH", "0.35")
    os.environ.setdefault("IOU_THRESH", "0.50")
    # Start the Flask development server.
    app.run(host=host, port=port, debug=False, threaded=True)

Writing /content/vehicle-counter/backend/app.py


### Mount Google Drive (Recommended)

To use your own video file without re-uploading it every time, you can mount your Google Drive. This cell will prompt you for authorization. Once mounted, your Drive files will be accessible from this notebook at the path `/content/drive/My Drive/`.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


After running the cell above and following the authorization steps, your Google Drive will be accessible at `/content/drive/My Drive/`. You can then place your video file in your Drive and update the `VIDEO_PATH` environment variable in the setup cell to point to its location in your Drive (e.g., `/content/drive/My Drive/your_video_folder/video1.mp4`).

## 2. Frontend Setup: HTML, CSS, and JavaScript

Now, let's create the files for the user interface.

### Frontend: HTML Structure

This cell writes the `index.html` file. It defines the structure of the web page, including the header, the video display area, the live metrics cards, and the control buttons.

In [5]:
%%writefile /content/vehicle-counter/frontend/index.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <title>Real‑Time Vehicle Counter (PoC)</title>
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap" rel="stylesheet" />
  <link rel="stylesheet" href="./styles.css" />
</head>
<body>
  <header class="app-header">
    <h1>Real‑Time Vehicle Counter</h1>
    <p class="subtitle">YOLOv8m + SORT / ByteTrack • Two‑way line crossing • Live counts</p>
  </header>

  <main class="container">
    <section class="video-card">
      <div class="video-header">
        <h2>Processed Stream</h2>
        <div class="status-bar" id="statusBar">
          <span class="dot" id="statusDot"></span>
          <span id="statusText">Connecting…</span>
        </div>
      </div>
      <div class="video-wrap">
        <div class="overlay" id="loadingOverlay">
          <div class="spinner"></div>
          <div>Connecting to video stream…</div>
        </div>
        <img id="video" alt="Live processed video stream" />
      </div>
      <div class="controls">
        <button id="btnReset" class="btn primary">Reset Counts</button>
        <button id="btnPause" class="btn">Pause</button>
        <button id="btnDownload" class="btn">Download CSV</button>
      </div>
    </section>

    <section class="counts-card">
      <h2>Live Metrics</h2>
      <div class="grid">
        <div class="metric up">
          <span class="label">Up</span>
          <span id="count-up" class="value">0</span>
          <span id="rate-up" class="hint">0.0 / min</span>
        </div>
        <div class="metric down">
          <span class="label">Down</span>
          <span id="count-down" class="value">0</span>
          <span id="rate-down" class="hint">0.0 / min</span>
        </div>
        <div class="metric total">
          <span class="label">Total</span>
          <span id="count-total" class="value">0</span>
          <span class="hint">Since last reset</span>
        </div>
        <div class="metric fps">
          <span class="label">Server FPS</span>
          <span id="fps" class="value">0</span>
          <span id="frames" class="hint">0 frames</span>
        </div>
      </div>
      <div class="meta">
        <span id="last-updated">Last updated: —</span>
      </div>
    </section>
  </main>

  <footer class="app-footer">
    <small>
      Decoupled front end • Set backend via <code>?backend=&lt;URL&gt;</code>.
      Current: <span id="backendUrl"></span>
    </small>
  </footer>

  <script src="./script.js"></script>
</body>
</html>

Writing /content/vehicle-counter/frontend/index.html


### Frontend: CSS Styling

This cell creates the `styles.css` file. It contains all the styling rules that control the visual appearance of our web application, such as colors, fonts, and layout, giving it a clean and modern look.

In [6]:
%%writefile /content/vehicle-counter/frontend/styles.css
:root {
  --bg: #0b111b;
  --card: #121a26;
  --ink: #e8eef8;
  --muted: #a7b1c2;
  --up: #4ade80;
  --down: #60a5fa;
  --total: #a78bfa;
  --pill: #1f2a3b;
  --border: #223247;
  --error: #ef4444;
  --ok: #10b981;
}
* { box-sizing: border-box; }
html, body { height: 100%; }
body {
  margin: 0; background: var(--bg); color: var(--ink);
  font-family: Inter, system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif;
}
.app-header { padding: 24px; text-align: center; }
.app-header h1 { margin: 0 0 6px; font-size: 28px; }
.subtitle { margin: 0; color: var(--muted); }
.container {
  max-width: 1100px; margin: 0 auto; padding: 12px 20px 40px;
  display: grid; grid-template-columns: 2fr 1fr; gap: 16px;
}
@media (max-width: 980px) { .container { grid-template-columns: 1fr; } }
.video-card, .counts-card {
  background: var(--card); border: 1px solid var(--border);
  border-radius: 12px; padding: 16px; box-shadow: 0 10px 30px rgba(0,0,0,0.25);
}
.video-header { display: flex; align-items: center; justify-content: space-between; margin-bottom: 10px; }
.video-header h2, .counts-card h2 { margin: 0; font-size: 18px; }
.status-bar {
  display: inline-flex; align-items: center; gap: 8px; background: var(--pill);
  border: 1px solid var(--border); border-radius: 999px; padding: 6px 10px; font-size: 12px; color: var(--muted);
}
.status-bar.ok { color: var(--ink); }
.status-bar.error { color: #ffd5d5; }
.dot { width: 10px; height: 10px; border-radius: 50%; background: var(--muted); }
.status-bar.ok .dot { background: var(--ok); animation: pulse 2s infinite; }
.status-bar.error .dot { background: var(--error); }
@keyframes pulse { 0% { box-shadow: 0 0 0 0 rgba(16,185,129,0.6);} 70% { box-shadow: 0 0 0 10px rgba(16,185,129,0);} 100% { box-shadow: 0 0 0 0 rgba(16,185,129,0);} }
.video-wrap { position: relative; width: 100%; background: #0c141f; border-radius: 8px; overflow: hidden; border: 1px solid var(--border); min-height: 240px; }
#video { display: block; width: 100%; height: auto; }
.overlay { position: absolute; inset: 0; background: rgba(0,0,0,0.75); display: flex; align-items: center; justify-content: center; color: #fff; font-size: 14px; gap: 10px; z-index: 2; }
.spinner { width: 22px; height: 22px; border-radius: 50%; border: 3px solid rgba(255,255,255,0.3); border-top-color: #fff; animation: spin .9s linear infinite; }
@keyframes spin { to { transform: rotate(360deg); } }
.controls { display: flex; gap: 10px; margin-top: 12px; flex-wrap: wrap; }
.btn { background: #1a2332; color: var(--ink); border: 1px solid var(--border); border-radius: 8px; padding: 10px 14px; font-weight: 600; font-size: 13px; cursor: pointer; }
.btn:hover { transform: translateY(-1px); }
.btn.primary { background: linear-gradient(135deg, #667eea, #764ba2); border: none; }
.counts-card .grid { display: grid; grid-template-columns: 1fr 1fr; gap: 12px; margin-top: 10px; }
@media (max-width: 520px) { .counts-card .grid { grid-template-columns: 1fr; } }
.metric { border: 1px solid var(--border); border-radius: 10px; padding: 16px; background: #0f1824; display: grid; grid-template-rows: auto auto auto; gap: 6px; }
.metric .label { color: var(--muted); font-size: 13px; }
.metric .value { font-size: 32px; font-weight: 700; letter-spacing: 1px; }
.metric .hint { color: var(--muted); font-size: 12px; }
.metric.up .value { color: var(--up); }
.metric.down .value { color: var(--down); }
.metric.total .value { color: var(--total); }
.meta { margin-top: 10px; color: var(--muted); font-size: 12px; text-align: right; }
.app-footer { text-align: center; padding: 22px 12px 32px; color: var(--muted); }

Writing /content/vehicle-counter/frontend/styles.css


### Frontend: JavaScript Logic

This cell writes the `script.js` file, which brings the user interface to life. This code is responsible for:
* Connecting to the backend's video stream and API endpoints.
* Periodically fetching and displaying the latest vehicle counts and metrics.
* Handling clicks on the "Reset," "Pause," and "Download CSV" buttons.

In [7]:
%%writefile /content/vehicle-counter/frontend/script.js
/**
 * Backend URL Configuration
 * --------------------------
 * This section determines the backend API URL to connect to.
 * It prioritizes in the following order:
 * 1. A 'backend' URL parameter (e.g., ?backend=http://192.168.1.10:5000). If found,
 * it's saved to localStorage and the page is reloaded without the parameter.
 * 2. A previously saved URL from localStorage.
 * 3. A default fallback URL (http://127.0.0.1:5000).
 */
const saved = localStorage.getItem("backend_url");
const param = new URL(window.location.href).searchParams.get("backend");
if (param) {
  // If a 'backend' URL parameter exists, save it for future visits.
  localStorage.setItem("backend_url", param);
  const u = new URL(window.location.href);
  u.searchParams.delete("backend"); // Clean the URL
  window.location.replace(u.toString()); // Reload the page with the clean URL
}
const BACKEND_BASE_URL = param || saved || "http://127.0.0.1:5000";

// --- DOM Element References ---
// Get and cache references to all the HTML elements we'll need to interact with.
const videoEl = document.getElementById("video");
const overlayEl = document.getElementById("loadingOverlay");
const statusBar = document.getElementById("statusBar");
const statusText = document.getElementById("statusText");
const upEl = document.getElementById("count-up");
const downEl = document.getElementById("count-down");
const totalEl = document.getElementById("count-total");
const rateUpEl = document.getElementById("rate-up");
const rateDownEl = document.getElementById("rate-down");
const fpsEl = document.getElementById("fps");
const framesEl = document.getElementById("frames");
const lastUpdatedEl = document.getElementById("last-updated");
const backendUrlEl = document.getElementById("backendUrl");
const btnReset = document.getElementById("btnReset");
const btnPause = document.getElementById("btnPause");
const btnDownload = document.getElementById("btnDownload");

// --- Constants and State Variables ---
// Configuration for update intervals.
const UPDATE_INTERVAL_MS = 1000; // How often to fetch new counts (1 second).
const HEALTH_INTERVAL_MS = 30000; // How often to check if the backend is alive (30 seconds).
const METRICS_INTERVAL_MS = 2000; // How often to fetch performance metrics like FPS (2 seconds).
const HISTORY_LIMIT = 900; // Maximum number of data points to store for the CSV download.

// Application state variables.
let isPaused = false; // Toggles whether the app is polling for new data.
let countsInFlight = false; // Flag to prevent multiple simultaneous requests for counts.
let metricsInFlight = false; // Flag for metrics requests.
let healthInFlight = false; // Flag for health check requests.
let previousCounts = { up: 0, down: 0 }; // Store the last known counts to detect changes.
let startTime = Date.now(); // Timestamp for when the app started, used for calculating rates.
let countHistory = []; // Array to store historical data for CSV export.

// --- Initial Setup ---
// Display the determined backend URL in the footer.
backendUrlEl.textContent = BACKEND_BASE_URL;
// Set the source of the video element to the backend's video feed endpoint.
videoEl.src = `${BACKEND_BASE_URL}/video_feed`;

/**
 * Updates the UI status indicator.
 * @param {boolean} ok - If true, the status is 'ok' (green). If false, it's 'error' (red).
 * @param {string} msg - The message to display (e.g., "Live", "Stream error").
 */
function setStatus(ok, msg) {
  statusText.textContent = msg;
  statusBar.classList.toggle("ok", !!ok);
  statusBar.classList.toggle("error", !ok);
}

// --- Video Element Event Handlers ---
// When the video stream loads successfully, hide the loading overlay and set status to "Live".
videoEl.onload = () => { overlayEl.style.display="none"; setStatus(true,"Live"); };
// If there's an error loading the video, show the overlay and an error message.
videoEl.onerror = () => { overlayEl.style.display="flex"; setStatus(false,"Stream error — check backend"); };

/**
 * Fetches and updates the vehicle counts from the backend API.
 */
async function refreshCounts() {
  // Don't fetch if paused or if another request is already in progress.
  if (isPaused || countsInFlight) return;
  countsInFlight = true;

  try {
    const res = await fetch(`${BACKEND_BASE_URL}/api/counts`, { cache: "no-store" });
    if (!res.ok) throw new Error(`HTTP ${res.status}`);
    const data = await res.json();

    // Update UI with the new counts.
    const up = Number(data.up ?? 0);
    const down = Number(data.down ?? 0);
    const total = up + down;
    animateValue(upEl, up, previousCounts.up);
    animateValue(downEl, down, previousCounts.down);
    totalEl.textContent = total.toString();

    // Calculate and display the rate (vehicles per minute).
    const elapsedMin = (Date.now() - startTime) / 60000;
    if (elapsedMin > 0) {
      rateUpEl.textContent = `${(up / elapsedMin).toFixed(1)} / min`;
      rateDownEl.textContent = `${(down / elapsedMin).toFixed(1)} / min`;
    }

    // Store history for CSV download and manage its size.
    countHistory.push({ ts: new Date().toISOString(), up, down, total });
    if (countHistory.length > HISTORY_LIMIT) {
      countHistory.splice(0, countHistory.length - HISTORY_LIMIT);
    }

    // Update state for the next cycle.
    previousCounts = { up, down };
    lastUpdatedEl.textContent = `Last updated: ${new Date().toLocaleTimeString()}`;
    if (statusBar.classList.contains("error")) setStatus(true, "Live"); // Restore status if it was in error.
  } catch (err) {
    console.error("Counts fetch failed:", err);
    setStatus(false, "Backend unreachable");
  } finally {
    countsInFlight = false; // Always reset the flag.
  }
}

/**
 * Fetches and updates performance metrics like FPS from the backend.
 */
async function refreshMetrics() {
  if (metricsInFlight) return;
  metricsInFlight = true;
  try {
    const res = await fetch(`${BACKEND_BASE_URL}/api/metrics`, { cache: "no-store" });
    if (!res.ok) return;
    const m = await res.json();
    fpsEl.textContent = (m.fps ?? 0).toString();
    framesEl.textContent = `${m.frames_processed ?? 0} frames`;
  } catch {} // Fail silently if metrics aren't available.
  finally {
    metricsInFlight = false;
  }
}

/**
 * Performs a health check to see if the backend is responsive.
 */
async function healthCheck() {
  if (healthInFlight) return;
  healthInFlight = true;
  try {
    const res = await fetch(`${BACKEND_BASE_URL}/health`, { cache: "no-store" });
    if (!res.ok) throw new Error(`HTTP ${res.status}`);
    const data = await res.json();
    if (data && data.ok) {
      if (statusBar.classList.contains("error")) setStatus(true, "Live");
    } else {
      setStatus(false, "Health check failed");
    }
  } catch (e) {
    setStatus(false, "Backend not responding");
  } finally {
    healthInFlight = false;
  }
}

// --- Polling Intervals ---
// Set up timers to repeatedly call the fetch functions.
setInterval(refreshCounts, UPDATE_INTERVAL_MS);
setInterval(refreshMetrics, METRICS_INTERVAL_MS);
setInterval(healthCheck, HEALTH_INTERVAL_MS);

// Immediately call the functions once on page load to populate data.
refreshCounts();
refreshMetrics();
healthCheck();

// --- Button Event Listeners ---
// Handle the "Reset Counts" button click.
btnReset.addEventListener("click", async () => {
  try {
    const res = await fetch(`${BACKEND_BASE_URL}/api/reset`);
    if (!res.ok) throw new Error(`HTTP ${res.status}`);
    // Reset front-end state and UI to match the backend.
    previousCounts = { up: 0, down: 0 };
    startTime = Date.now();
    countHistory = [];
    upEl.textContent="0"; downEl.textContent="0"; totalEl.textContent="0";
    rateUpEl.textContent="0.0 / min"; rateDownEl.textContent="0.0 / min";
    setStatus(true, "Counts reset");
  } catch (e) {
    setStatus(false, "Reset failed");
  }
});

// Handle the "Pause" / "Resume" button click.
btnPause.addEventListener("click", () => {
  isPaused = !isPaused;
  btnPause.textContent = isPaused ? "Resume" : "Pause";
  setStatus(true, isPaused ? "Paused (polling stopped)" : "Live");
  // If resuming, immediately fetch the latest data.
  if (!isPaused) {
    refreshCounts();
    refreshMetrics();
  }
});

// Handle the "Download CSV" button click.
btnDownload.addEventListener("click", () => {
  if (!countHistory.length) {
    // Using a custom modal or message would be better than alert in a real app.
    // For this PoC, alert is simple and effective.
    alert("No data to download yet.");
    return;
  }
  // Convert the history array to a CSV string.
  let csv = "Timestamp,Up,Down,Total\n";
  countHistory.forEach(r => { csv += `${r.ts},${r.up},${r.down},${r.total}\n`; });

  // Create a blob and trigger a download.
  const blob = new Blob([csv], { type: "text/csv" });
  const url = URL.createObjectURL(blob);
  const a = document.createElement("a");
  a.href = url;
  a.download = `vehicle_counts_${new Date().toISOString()}.csv`;
  a.click(); // Programmatically click the link to start the download.
  URL.revokeObjectURL(url); // Clean up the created URL.
});

/**
 * Animates a number change in an element by briefly scaling it up.
 * @param {HTMLElement} el - The element to animate.
 * @param {number} next - The new value.
 * @param {number} prev - The previous value.
 */
function animateValue(el, next, prev) {
  // Only animate if the value has actually changed.
  if (next !== prev) {
    el.textContent = next.toString();
    el.classList.add("pulse");
    setTimeout(() => el.classList.remove("pulse"), 250); // Remove class after animation.
  }
}

// --- Dynamic CSS Injection ---
// Inject the 'pulse' animation style directly into the document's head.
// This keeps the JavaScript self-contained without needing a separate CSS file for this small effect.
const style = document.createElement("style");
style.textContent = `.pulse { transform: scale(1.04); transition: transform .2s; }`;
document.head.appendChild(style);

Writing /content/vehicle-counter/frontend/script.js


## 3. Install Dependencies

Before we can run the backend, we need to install the necessary Python libraries. This cell uses `pip` to install `ultralytics` (for YOLOv8), `Flask` (the web server), `OpenCV` (for video processing), and other required packages.

In [8]:
%pip install -q --upgrade pip setuptools wheel packaging
%pip uninstall -y -q ultralytics opencv-python opencv-contrib-python opencv-python-headless || true

# Core deps
%pip install -q "numpy>=2.1" "scipy>=1.14.1" "jedi"
%pip install -q "opencv-python-headless>=4.12.0.88"

# Ultralytics + Flask + CORS + SORT deps + ngrok
%pip install -q ultralytics flask==3.0.3 flask-cors==4.0.1 filterpy pyngrok

# (Optional) If Torch isn’t present or mismatched, uncomment ONE of the following:
# CPU-only:
# %pip install -q torch --index-url https://download.pytorch.org/whl/cpu
# CUDA 12.x (common on Colab):
# %pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, which is not installed.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
dopamine-rl 4.1.2 requires opencv-python>=3.4.8.29, which is not installed.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.3 which is incompatible.
cupy

### Verify Installations

This is a quick check to confirm that the key libraries were installed correctly and to print their versions. This can be useful for debugging potential version conflicts.

In [9]:
import importlib, cv2, torch, numpy, scipy
print("NumPy:", numpy.__version__)
print("SciPy:", scipy.__version__)
print("OpenCV:", cv2.__version__)
print("Torch:", torch.__version__)
print("Ultralytics:", importlib.import_module("ultralytics").__version__)


NumPy: 2.0.2
SciPy: 1.16.2
OpenCV: 4.12.0
Torch: 2.8.0+cu126
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics: 8.3.203


## 4. Configure the Application

This is the **main configuration cell**. Before running the application, you should review and adjust these settings. The most important ones are:
* `VIDEO_PATH`: **You must set this** to the correct path of your video file (e.g., in your mounted Google Drive).
* `LINE_Y`: The vertical position (in pixels) of the counting line.
* `TRACKER_MODE`: Choose between `SORT` and `BYTE` trackers.
* `RESIZE_WIDTH`: A smaller width (e.g., 640) can significantly speed up processing.

In [10]:
# ============================
# Environment setup for app.py
# ============================

import os

# ---- Required / commonly changed settings ----

# Path to the input video file that YOLO will read frame-by-frame.
# Tip: upload a file to Colab and set this to its path.
os.environ["VIDEO_PATH"] = "/content/drive/MyDrive/Vehicle-counter/video1.mp4"

# Vertical pixel position of the counting line.
# Objects crossing this horizontal line are counted as "up" or "down".
os.environ["LINE_Y"] = "360"

# Resize width for each frame before detection (smaller = faster, but less detail).
# Keep aspect ratio; height is computed automatically.
os.environ["RESIZE_WIDTH"] = "960"

# What to do when the video loops back to the start:
# - "true"  → reset counts when the video restarts
# - "false" → keep counts across loops (good for demos)
os.environ["RESET_ON_LOOP"] = "false"

# Which tracker to use:
# - "SORT" → uses our custom SORT implementation (fast, simple)
# - "BYTE" → uses Ultralytics' ByteTrack via model.track()
os.environ["TRACKER_MODE"] = "SORT"

# Host/port for the Flask server.
# In Colab, keep host "0.0.0.0". If you use ngrok, it will tunnel to this port.
os.environ["HOST"] = "0.0.0.0"
os.environ["PORT"] = "5000"


# ---- Optional overrides (these already match app.py defaults) ----
# Change them only if you want different behavior.

# YOLO model weights file. If not found locally, Ultralytics will download it.
os.environ["MODEL_PATH"] = "yolov8s.pt"

# Device selection:
# - "auto" → use CUDA GPU if available, otherwise CPU
# - "cuda" → force GPU (fails if no GPU)
# - "cpu"  → force CPU
os.environ["MODEL_DEVICE"] = "auto"

# Detection confidence threshold (higher = fewer, more confident detections).
os.environ["CONF_THRESH"] = "0.35"

# IoU threshold for NMS/association (controls how much boxes can overlap).
os.environ["IOU_THRESH"] = "0.50"

# How many recent center points to keep for drawing each track's trail.
os.environ["TRAIL_LEN"] = "20"

# JPEG quality for the MJPEG stream (0–100). Higher = better quality, larger bandwidth.
os.environ["JPEG_QUALITY"] = "80"


# ---- (Optional) quick sanity printout ----
print("Video:", os.environ["VIDEO_PATH"])
print("Mode:", os.environ["TRACKER_MODE"])
print("Device:", os.environ["MODEL_DEVICE"])
print("Serving at http://localhost:" + os.environ["PORT"])


Video: /content/drive/MyDrive/Vehicle-counter/video1.mp4
Mode: SORT
Device: auto
Serving at http://localhost:5000


## 5. Run the Application

With everything set up, let's launch the servers.

### Launch the Backend Server

This command starts the Flask web server (`app.py`) in the background. The `nohup` command ensures it keeps running, and its output is redirected to a log file named `backend.log`. We then display the end of the log file to check for any immediate errors.

In [11]:
%%bash
# ============================
# Launch the Flask backend in the background and show recent logs
# ============================

# 1) Move into the backend folder that contains app.py
cd /content/vehicle-counter/backend

# 2) Start the server in the background:
#    - `nohup` lets it keep running even if the notebook cell finishes
#    - `python app.py` runs your Flask app
#    - `> backend.log 2>&1` sends both stdout and stderr to backend.log
#    - `&` runs it in the background so the cell doesn’t block
nohup python app.py > backend.log 2>&1 &

# 3) Give the server a moment to start (download model, bind to port, etc.)
sleep 2

# 4) Show the last 30 lines of the log so you can confirm it’s running
#    - `|| true` prevents the cell from failing if the file is empty or missing
tail -n 30 backend.log || true

### Expose the Backend with a Public URL

The Flask server is running inside the Colab environment, which isn't directly accessible from the internet. This cell downloads and runs **Cloudflare Tunnel** to create a secure, temporary public URL that forwards to our backend server on port 5000.

**The URL generated by this cell is your backend endpoint.**

In [12]:
%%bash
cd /content
# Download cloudflared (no account needed)
wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
chmod +x cloudflared
# Open a tunnel to the backend on port 5000
nohup ./cloudflared tunnel --url http://localhost:5000 --no-autoupdate > cf_backend.log 2>&1 &
sleep 5 # Increased sleep time
echo "Backend tunnel logs (showing URL):"
grep -o "https://.*trycloudflare.com" -m 1 cf_backend.log || tail -n 50 cf_backend.log

Backend tunnel logs (showing URL):
https://privilege-identifying-formatting-novel.trycloudflare.com


### Launch the Frontend and Expose Its Public URL

Next, we need to serve our HTML, CSS, and JS files.
1.  A simple Python HTTP server is started in the `frontend` directory.
2.  Cloudflare Tunnel is used again to create a second public URL for this frontend server.

**This is the main URL you should open in your browser to view the application.**

In [13]:
%%bash
cd /content/vehicle-counter/frontend
nohup python -m http.server 8000 > fe.log 2>&1 &
sleep 2
cd /content
nohup ./cloudflared tunnel --url http://localhost:8000 --no-autoupdate > cf_frontend.log 2>&1 &
sleep 5 # Increased sleep time
echo "Frontend tunnel logs (showing URL):"
grep -o "https://.*trycloudflare.com" -m 1 cf_frontend.log || tail -n 50 cf_frontend.log

Frontend tunnel logs (showing URL):
https://lan-relative-motor-arthritis.trycloudflare.com


## 6. Access Your Application

The final step is to launch the web interface. The code cell below automates this process by doing the following:

1.  It reads the log files that were created when we launched the public tunnels.
2.  It automatically finds and extracts the unique public URLs for both the frontend and backend servers.
3.  It combines these two URLs into a single, final link, correctly passing the backend address to the frontend.

A styled, clickable button will appear in the output below. **Click this link to open your application in a new browser tab.**

In [14]:
import re
import os
from IPython.display import display, HTML

# --- Configuration ---
BACKEND_LOG_FILE = '/content/cf_backend.log'
FRONTEND_LOG_FILE = '/content/cf_frontend.log'

# --- Function to extract URL from a log file ---
def find_url_in_log(log_file):
    """Searches a log file for the first Cloudflare URL and returns it."""
    if not os.path.exists(log_file):
        return None
    try:
        with open(log_file, 'r') as f:
            for line in f:
                # Use regex to find the URL
                match = re.search(r'(https?://[a-zA-Z0-9-]+\.trycloudflare\.com)', line)
                if match:
                    return match.group(1)
    except Exception as e:
        print(f"Error reading {log_file}: {e}")
    return None

# --- Main script ---
backend_url = find_url_in_log(BACKEND_LOG_FILE)
frontend_url = find_url_in_log(FRONTEND_LOG_FILE)

print("--- URL Extraction ---")
print(f"Backend Log: {BACKEND_LOG_FILE}")
print(f"Frontend Log: {FRONTEND_LOG_FILE}")
print("-" * 20)

if backend_url:
    print(f"✅ Backend URL Found: {backend_url}")
else:
    print(f"❌ Backend URL not found. Check '{BACKEND_LOG_FILE}' for errors.")

if frontend_url:
    print(f"✅ Frontend URL Found: {frontend_url}")
else:
    print(f"❌ Frontend URL not found. Check '{FRONTEND_LOG_FILE}' for errors.")

print("-" * 20)

# --- Display the final clickable link ---
if backend_url and frontend_url:
    final_url = f"{frontend_url}/?backend={backend_url}"

    # Create an HTML link with improved styling for better visibility
    link_html = (
        f'<div style="font-family: -apple-system, BlinkMacSystemFont, \'Segoe UI\', Roboto, Helvetica, Arial, sans-serif; max-width: 600px; margin: 25px auto; padding: 25px; border-radius: 12px; border: 1px solid #e0e0e0; background: linear-gradient(135deg, #f9f9f9, #ffffff); box-shadow: 0 10px 25px rgba(0,0,0,0.08); text-align: center;">'
        f'<h2 style="color: #2c3e50; font-size: 24px; margin-bottom: 10px;"> Your Application is Ready!</h2>'
        f'<p style="color: #555; font-size: 16px; margin-bottom: 25px;">Click the button below to open the Vehicle Counter:</p>'
        f'<a href="{final_url}" target="_blank" style="display: inline-block; font-size: 18px; font-weight: bold; color: #ffffff; background: linear-gradient(135deg, #3498db, #2980b9); padding: 14px 28px; border-radius: 8px; text-decoration: none; box-shadow: 0 5px 15px rgba(52, 152, 219, 0.4); transition: all 0.3s ease;">'
        f'Open Application'
        f'</a>'
        f'<p style="font-size: 12px; margin-top: 20px; color: #7f8c8d;">'
        f'Or copy this URL:'
        f'<br><code style="background-color: #ecf0f1; padding: 4px 8px; border-radius: 4px; color: #2c3e50; word-break: break-all; display: inline-block; margin-top: 5px;">{final_url}</code>'
        f'</p>'
        f'</div>'
    )

    display(HTML(link_html))
else:
    error_html = (
        '<div style="font-family: -apple-system, BlinkMacSystemFont, \'Segoe UI\', Roboto, Helvetica, Arial, sans-serif; max-width: 600px; margin: 25px auto; padding: 25px; border-radius: 12px; border: 1px solid #f5c6cb; background: linear-gradient(135deg, #fff5f5, #ffebeb); box-shadow: 0 10px 25px rgba(0,0,0,0.08); text-align: center;">'
        '<h2 style="color: #721c24; font-size: 24px;">⚠️ Could Not Generate Link</h2>'
        '<p style="color: #721c24; font-size: 16px;">One or both of the necessary URLs could not be found. Please scroll up and check the output of the cells that run the Cloudflare Tunnels for any errors.</p>'
        '</div>'
    )
    display(HTML(error_html))



--- URL Extraction ---
Backend Log: /content/cf_backend.log
Frontend Log: /content/cf_frontend.log
--------------------
✅ Backend URL Found: https://privilege-identifying-formatting-novel.trycloudflare.com
✅ Frontend URL Found: https://lan-relative-motor-arthritis.trycloudflare.com
--------------------
